<a href="https://colab.research.google.com/github/EkaterinaMikheeva/Other_projects/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B5%D1%80_%D0%B2%D0%B0%D0%BA%D0%B0%D0%BD%D1%81%D0%B8%D0%B9_HH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Описание задачи: Создать парсер вакансий на сайте HH. 

Парсер для себя  

Условия отбора в парсинге :

1. Не интересует геймдев
2. По опыту просмотра hh  - все что касается бизнес - аналитиков - это не про меня. Поэтому "бизнес" - тоже стоп слово
3. Удаленная работа

Про опыт: я не буду ставить какие то ограничения, так как технчески у меня 13 лет в маркетинге (разные сферы) и я планирую развиваться именно в продуктовой или ecom аналитике. Но как доп поле оно полезно



```



### Импорт библиотек

In [ ]:
import requests
import pandas as pd
import numpy as np

import json

from datetime import datetime, timedelta
import time
import os
from IPython.core.display import display, HTML, clear_output
import ipywidgets as widgets
from typing import Union

import matplotlib.pyplot as plt
import seaborn as sns


## Делаем выгрузку из HeadHunter





```
# Выбран кодовый формат
```


In [ ]:
BASE_URL = "https://api.hh.ru/"

TEXT = f'SQL NAME:("Аналитик" or "Analyst" or "DWH" or "BI" or "Business Intelligence" or "ETL" or "КХД")'

# стоп слова в NAME: разработчик

#TEXT = f'SQL NAME:(("Аналитик" or "Analyst" or "DWH" or "BI" or "Business Intelligence" or "ETL" or "КХД )and (not "бизнес" not "Разработчик") ) AND DESCRIPTION: (NOT "гейм" )'

dt_now = datetime.now().date()
dt_from = dt_now - timedelta(days=7)
DATE_TO = dt_now.isoformat() #API требует дату в формате ISO 8601 YYYY-MM-DD или YYYY-MM-DDThh:mm:ss±hhmm
DATE_FROM = dt_from.isoformat()

VAC_URL = BASE_URL + f'vacancies?text={TEXT}&date_from={DATE_FROM}&date_to={DATE_TO}&responses_count_enabled=True&per_page=100'

In [ ]:
vacancy_df = pd.DataFrame() 


In [ ]:
experiences = ['noExperience',
               'between1And3',
               'between3And6', 
               'moreThan6']

#### Напишем цикл для направления запросов к API

In [ ]:
for exp in experiences:
  page=0
  url = VAC_URL + f'&experience = {exp}'
  try:
    r = requests.get(url+ f'&page={page}')
    data = json.loads(r.text)
    print(exp, ' - ', data['pages'])
    items=data['items']
    for page in range (1,data['pages'] ):
      r = requests.get(url+ f'&page={page}')
      data = json.loads(r.text)
      items += data['items']
    df = pd.DataFrame(items)
    df['experience'] = exp
    vacancy_df = pd.concat([vacancy_df, df])
  except Exception as e:
    print (exp, '-', e)
    time.sleep(10)

noExperience  -  18
between1And3  -  18
between3And6  -  18
moreThan6  -  18


### Вытащим ценную информацию из словарей

In [ ]:
vacancy_df

,experience,id,premium,name,department,has_test,response_letter_required,area,salary,type,...,relations,employer,snippet,contacts,schedule,counters,working_days,working_time_intervals,working_time_modes,accept_temporary
0,noExperience,54379864,False,Разработчик хранилищ данных,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '1647148', 'name': 'Клевертек', 'url': ...",{'requirement': 'Опыт в проектировании и разра...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 20},[],[],[],False
1,noExperience,50115694,False,Аналитик BI (отчетность DWH),None,False,False,"{'id': '54', 'name': 'Красноярск', 'url': 'htt...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '78638', 'name': 'Тинькофф', 'url': 'ht...",{'requirement': 'Уверенный <highlighttext>SQL<...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",{'responses': 2},[],[],[],False
2,noExperience,54372168,False,Разработчик хранилищ данных,None,False,False,"{'id': '1002', 'name': 'Минск', 'url': 'https:...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '2731087', 'name': 'Клевер Технолоджи',...",{'requirement': 'Опыт в проектировании и разра...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 20},[],[],[],False
3,noExperience,54520888,False,Аналитик BI,None,False,False,"{'id': '77', 'name': 'Рязань', 'url': 'https:/...","{'from': 57000, 'to': None, 'currency': 'RUR',...","{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '1744862', 'name': 'Email Soldiers', 'u...",{'requirement': 'знания функционала с MS Power...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",{'responses': 2},[],[],[],False
4,noExperience,53549769,False,Системный аналитик DWH,None,False,False,"{'id': '160', 'name': 'Алматы', 'url': 'https:...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '78638', 'name': 'Тинькофф', 'url': 'ht...",{'requirement': 'Свободно владеешь <highlightt...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",{'responses': 11},[],[],[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,moreThan6,49528911,False,Специалист по внедрению и сопровождению КИС (А...,None,False,False,"{'id': '2', 'name': 'Санкт-Петербург', 'url': ...","{'from': 100000, 'to': None, 'currency': 'RUR'...","{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '2451337', 'name': 'Машиностроительный ...",{'requirement': 'Опыт администрирования различ...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 4},[],[],[],False
1792,moreThan6,49791229,False,Программист - архитектор баз данных (Хранилище...,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '53417', 'name': 'Henderson', 'url': 'h...",{'requirement': 'Обязательные знания: Понимани...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 2},[],[],[],False
1793,moreThan6,53622567,False,Программист - архитектор баз данных (Хранилище...,None,False,False,"{'id': '2077', 'name': 'Химки', 'url': 'https:...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '53417', 'name': 'Henderson', 'url': 'h...",{'requirement': 'Обязательные знания: Понимани...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 0},[],[],[],False
1794,moreThan6,54634644,False,Ведущий риск-аналитик Группа по анализу рисков...,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '6591', 'name': 'Промсвязьбанк', 'url':...",{'requirement': '...материалов для принятия ре...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 9},[],[],[],False


In [ ]:
vacancy_df.reset_index(drop = True, inplace = True)

In [ ]:
vacancy_df.head()

,experience,id,premium,name,department,has_test,response_letter_required,area,salary,type,...,relations,employer,snippet,contacts,schedule,counters,working_days,working_time_intervals,working_time_modes,accept_temporary
0,noExperience,54379864,False,Разработчик хранилищ данных,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '1647148', 'name': 'Клевертек', 'url': ...",{'requirement': 'Опыт в проектировании и разра...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 20},[],[],[],False
1,noExperience,50115694,False,Аналитик BI (отчетность DWH),None,False,False,"{'id': '54', 'name': 'Красноярск', 'url': 'htt...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '78638', 'name': 'Тинькофф', 'url': 'ht...",{'requirement': 'Уверенный <highlighttext>SQL<...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",{'responses': 2},[],[],[],False
2,noExperience,54372168,False,Разработчик хранилищ данных,None,False,False,"{'id': '1002', 'name': 'Минск', 'url': 'https:...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '2731087', 'name': 'Клевер Технолоджи',...",{'requirement': 'Опыт в проектировании и разра...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 20},[],[],[],False
3,noExperience,54520888,False,Аналитик BI,None,False,False,"{'id': '77', 'name': 'Рязань', 'url': 'https:/...","{'from': 57000, 'to': None, 'currency': 'RUR',...","{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '1744862', 'name': 'Email Soldiers', 'u...",{'requirement': 'знания функционала с MS Power...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",{'responses': 2},[],[],[],False
4,noExperience,53549769,False,Системный аналитик DWH,None,False,False,"{'id': '160', 'name': 'Алматы', 'url': 'https:...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],"{'id': '78638', 'name': 'Тинькофф', 'url': 'ht...",{'requirement': 'Свободно владеешь <highlightt...,None,"{'id': 'remote', 'name': 'Удаленная работа'}",{'responses': 11},[],[],[],False


In [ ]:
# напишем функцию:
def return_id (x, key = 'id', nan_value = np.nan):
  try:
    return x[key]
  except Exception as e:
    return nan_value
  

In [ ]:
vacancy_df['employer_name'] = vacancy_df['employer'].apply(return_id, key = 'name')

vacancy_df['department_name'] = vacancy_df['department'].apply(return_id, key = 'name')
vacancy_df['salary_from'] = vacancy_df['salary'].apply(return_id, key = 'from')
vacancy_df['salary_from'] = vacancy_df['salary'].apply(return_id, key = 'from')
vacancy_df['salary_to'] = vacancy_df['salary'].apply(return_id, key = 'to')
vacancy_df['salary_gross'] = vacancy_df['salary'].apply(return_id, key = 'gross')
vacancy_df['salary_currency'] = vacancy_df['salary'].apply(return_id, key = 'currency')
vacancy_df['city'] = vacancy_df['area'].apply(return_id, key = 'name')

In [ ]:
vacancy_df.head()

,experience,id,premium,name,department,has_test,response_letter_required,area,salary,type,...,working_time_modes,accept_temporary,employer_name,department_name,salary_from,salary_to,salary_gross,salary_currency,cuty,city
0,noExperience,54379864,False,Разработчик хранилищ данных,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],False,Клевертек,NaN,NaN,NaN,NaN,NaN,Москва,Москва
1,noExperience,50115694,False,Аналитик BI (отчетность DWH),None,False,False,"{'id': '54', 'name': 'Красноярск', 'url': 'htt...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],False,Тинькофф,NaN,NaN,NaN,NaN,NaN,Красноярск,Красноярск
2,noExperience,54372168,False,Разработчик хранилищ данных,None,False,False,"{'id': '1002', 'name': 'Минск', 'url': 'https:...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],False,Клевер Технолоджи,NaN,NaN,NaN,NaN,NaN,Минск,Минск
3,noExperience,54520888,False,Аналитик BI,None,False,False,"{'id': '77', 'name': 'Рязань', 'url': 'https:/...","{'from': 57000, 'to': None, 'currency': 'RUR',...","{'id': 'open', 'name': 'Открытая'}",...,[],False,Email Soldiers,NaN,57000.0,NaN,True,RUR,Рязань,Рязань
4,noExperience,53549769,False,Системный аналитик DWH,None,False,False,"{'id': '160', 'name': 'Алматы', 'url': 'https:...",None,"{'id': 'open', 'name': 'Открытая'}",...,[],False,Тинькофф,NaN,NaN,NaN,NaN,NaN,Алматы,Алматы


### Про дубликаты

In [ ]:
vacancy_df[vacancy_df.duplicated(['employer_name', 'name', 'city'], keep = False)][
['employer_name', 'name', 'city', 'experience']].sort_values(by='name')

,employer_name,name,city,experience
651,DATS.TEAM,Ad-hoc Аналитик,Москва,noExperience
2448,DATS.TEAM,Ad-hoc Аналитик,Москва,between1And3
6041,DATS.TEAM,Ad-hoc Аналитик,Москва,moreThan6
4245,DATS.TEAM,Ad-hoc Аналитик,Москва,between3And6
6262,ЦМД-софт,Analyst / Аналитик,Москва,moreThan6
...,...,...,...,...
6158,Аналитический центр при Правительстве Российск...,Эксперт-аналитик (мониторинг достижения национ...,Москва,moreThan6
1704,БАНК УРАЛСИБ,Эксперт-аналитик развития системы Way4,Уфа,noExperience
3500,БАНК УРАЛСИБ,Эксперт-аналитик развития системы Way4,Уфа,between1And3
5297,БАНК УРАЛСИБ,Эксперт-аналитик развития системы Way4,Уфа,between3And6


## Срез по интересным компаниям

In [ ]:
vacancy_df.dropna(subset = ['employer_name', 'salary_from'])[vacancy_df['employer_name'].str.lower().str.contains('яндекс')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,experience,id,premium,name,department,has_test,response_letter_required,area,salary,type,...,working_time_modes,accept_temporary,employer_name,department_name,salary_from,salary_to,salary_gross,salary_currency,cuty,city
690,noExperience,54428168,False,Ревьюер проектов на курс по Data Science и Ана...,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 70000, 'to': 150000, 'currency': 'RUR...","{'id': 'open', 'name': 'Открытая'}",...,[],False,Яндекс.Практикум,NaN,70000.0,150000.0,True,RUR,Москва,Москва
2487,between1And3,54428168,False,Ревьюер проектов на курс по Data Science и Ана...,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 70000, 'to': 150000, 'currency': 'RUR...","{'id': 'open', 'name': 'Открытая'}",...,[],False,Яндекс.Практикум,NaN,70000.0,150000.0,True,RUR,Москва,Москва
4284,between3And6,54428168,False,Ревьюер проектов на курс по Data Science и Ана...,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 70000, 'to': 150000, 'currency': 'RUR...","{'id': 'open', 'name': 'Открытая'}",...,[],False,Яндекс.Практикум,NaN,70000.0,150000.0,True,RUR,Москва,Москва
6080,moreThan6,54428168,False,Ревьюер проектов на курс по Data Science и Ана...,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': 70000, 'to': 150000, 'currency': 'RUR...","{'id': 'open', 'name': 'Открытая'}",...,[],False,Яндекс.Практикум,NaN,70000.0,150000.0,True,RUR,Москва,Москва


## Ищем вакансии

In [ ]:
city_vacs = vacancy_df.query('(city == "Москва") or (city == "Алмата")')    
# релокация- релокация) 

In [ ]:
vacs_df = city_vacs.copy().reset_index(drop=True).query("experience in ['noExperience', 'between1And3']")

In [ ]:
vacs_df['url'].head()

0    https://api.hh.ru/vacancies/54379864?host=hh.ru
1    https://api.hh.ru/vacancies/54687106?host=hh.ru
2    https://api.hh.ru/vacancies/54692868?host=hh.ru
3    https://api.hh.ru/vacancies/54622562?host=hh.ru
4    https://api.hh.ru/vacancies/52400488?host=hh.ru
Name: url, dtype: object

In [ ]:
vacs_df.shape

(2081, 38)

С помощью библиотек json и requests получим нужную информацию, добавим магии 


In [ ]:
HTML(json.loads(requests.get('https://api.hh.ru/vacancies/54379864?host=hh.ru').text)['description'])

Сохраним информацию в читаемом виде (  и тут все зафисло на 10 минут) 
Не понятно из-за чего? может, потому что город Москва и много вакансий? 

In [ ]:
vacs = vacs_df['url'].apply(lambda x: HTML(json.loads(requests.get(x).text)['description']))

In [ ]:
vacs

0       <IPython.core.display.HTML object>
1       <IPython.core.display.HTML object>
2       <IPython.core.display.HTML object>
3       <IPython.core.display.HTML object>
4       <IPython.core.display.HTML object>
                       ...                
2076    <IPython.core.display.HTML object>
2077    <IPython.core.display.HTML object>
2078    <IPython.core.display.HTML object>
2079    <IPython.core.display.HTML object>
2080    <IPython.core.display.HTML object>
Name: url, Length: 2081, dtype: object

### Добавим кнопку

In [ ]:
button = widgets.Button(description = 'Click_me')
output = widgets.Output()

display(button,output)

counter= 0
def on_clicked(b):
  with output:
    try:
      global counter
      clear_output()
      print ('№ Вакансии:', counter)
      display(HTML ("<h1>" + str (vacs_df['name'][counter]) + ' в '+ str (vacs_df['employer_name'][counter]) +"</h1>" ))
      display(vacs[counter])
      display(HTML ('<a href="' + str (vacs_df['alternate_url'][counter]) + '"target="_blank">Ссылка на вакансию</a>'))
      counter +=1
    except Exception as e:
      print ('Вакансии закончились')


button.on_click(on_clicked)

Button(description='Click_me', style=ButtonStyle())

Output()

In [ ]:
from bs4 import BeautifulSoup

vacs  = vacs_df['url'].apply(lambda x: BeautifulSoup(json.loads(requests.get(x).text)['description']).get_text())

KeyboardInterrupt: ignored